# Exploring the Allen Brain Observatory Database
In this notebook, we'll explore data from the allen Brain Observatory database. See their [documentation](http://help.brain-map.org/display/observatory/Documentation) for more details about the mice, experimental setup, data processing, and analysis.

For an example notebook from the Allen Institute that demonstrates how to work with the data, see [here](http://alleninstitute.github.io/AllenSDK/_static/examples/nb/brain_observatory.html).

### Two recent preprints that have made use of this data:
1. Christensen and Pillow (2017). [Running reduces firing but improves coding in rodent higher-order visual cortex](https://www.biorxiv.org/content/early/2017/11/04/214007)
2. Esfahany et al. (2017). [Organization of Neural Population Code in Mouse Visual System](https://www.biorxiv.org/content/early/2017/12/04/220558)

### Also see
* [This helpful blog post](https://kachio.github.io/blog/2016/12/10/Decoding-Identity-of-Natural-Images) on wrangling the Brain Observatory datasets and natural imaging decoding from Onyekachi Odoemene in Anne Churchland's lab. Some of the code for importing is modified from this post, as well as the AllenSDk example.


In [1]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from allensdk.brain_observatory.natural_scenes import NaturalScenes
import numpy as np
import h5py
import os
# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

In [2]:
import pprint
targeted_structures = boc.get_all_targeted_structures()
print("all targeted structures: " + str(targeted_structures))

cre_lines = boc.get_all_cre_lines()
print("all cre lines: " + str(cre_lines))

all targeted structures: [u'VISal', u'VISam', u'VISl', u'VISp', u'VISpm', u'VISrl']
all cre lines: [u'Cux2-CreERT2', u'Emx1-IRES-Cre', u'Nr5a1-Cre', u'Rbp4-Cre_KL100', u'Rorb-IRES2-Cre', u'Scnn1a-Tg3-Cre']


In [3]:
experiments = boc.get_ophys_experiments(stimuli=['natural_scenes'],
                                        targeted_structures =['VISl', 'VISp'],
                                        session_types=[stim_info.THREE_SESSION_B],
                                       cre_lines=['Cux2-CreERT2'])
len(experiments)

29

In [ ]:
for i,exp in enumerate(experiments):
    print(i)
    boc.get_ophys_experiment_data(exp['id']) 

In [5]:
import pandas as pd

# The following line takes a while to run the first time
cells = boc.get_cell_specimens()
cells = pd.DataFrame.from_records(cells)
print("total cells: %d" % len(cells))

total cells: 37091


In [6]:
cells

,all_stim,area,cell_specimen_id,donor_full_genotype,dsi_dg,experiment_container_id,failed_experiment_container,g_dsi_dg,g_osi_dg,g_osi_sg,...,specimen_id,tfdi_dg,time_to_peak_ns,time_to_peak_sg,tld1_id,tld1_name,tld2_id,tld2_name,tlr1_id,tlr1_name
0,False,VISpm,517394847,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,0.852081,511498500,False,0.468690,0.275754,NaN,...,503292439,0.348821,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
1,False,VISpm,517394850,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,0.730537,0.342070,NaN,...,503292439,0.298012,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
2,False,VISpm,517394854,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,0.692137,511498500,False,0.319382,0.215510,NaN,...,503292439,0.380637,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
3,False,VISpm,517394858,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,NaN,NaN,0.761230,...,503292439,NaN,0.46550,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
4,False,VISpm,517394862,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,0.391861,0.742900,NaN,...,503292439,0.330543,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
5,False,VISpm,517394882,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,NaN,NaN,NaN,...,503292439,NaN,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
6,False,VISpm,517394890,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,NaN,NaN,0.670796,...,503292439,NaN,0.33250,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
7,False,VISpm,517394898,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,0.236221,511498500,False,0.266105,0.289409,NaN,...,503292439,0.377737,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
8,False,VISpm,517394902,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,NaN,NaN,0.993565,...,503292439,NaN,0.26600,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
9,False,VISpm,517394906,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,0.656459,511498500,False,0.665352,0.905118,NaN,...,503292439,0.226531,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
